Построить классификатор, используя сверточную нейронную сеть на датасете CIFAR-10 с применением не менее двух разных вариантов настройки:

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
train_data = dsets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = dsets.CIFAR10(root='./data', train=False, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
batch_size = 100
num_workers = 0
num_epochs = 5

In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [6]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = 10

In [7]:
#Defining the convolutional neural network
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(6), # nn.LayerNorm([6,28,28])
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16), # nn.LayerNorm([16,10,10]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(16*8*8, 120)
        self.dropout = nn.Dropout(0.25)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)



    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
model = CNN(num_classes)

In [9]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1024, out_features=120, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (relu): ReLU()
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [10]:
learning_rate = 0.01
#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [11]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, loss.item()))

Epoch [1/5], Loss: 1.6542
Epoch [2/5], Loss: 1.3851
Epoch [3/5], Loss: 1.2125
Epoch [4/5], Loss: 1.1120
Epoch [5/5], Loss: 1.4014


In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 55.13 %


In [13]:
#Defining the convolutional neural network
import torch.nn as nn

class CNN2(nn.Module):
    def __init__(self, num_classes):
        super(CNN2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            # nn.BatchNorm2d(32), # nn.LayerNorm([6,28,28])
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        ) # 64 * 16 * 16
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(64), # nn.LayerNorm([16,10,10]),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        ) # 128 * 8 * 8

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) # 256 * 4 * 4
        self.fc = nn.Linear(256*4*4, 1024)
        self.dropout = nn.Dropout(0.25)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(1024, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)



    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [14]:
model5 = CNN2(num_classes)
model5

CNN2(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-

In [15]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, loss.item()))

Epoch [1/5], Loss: 0.9966
Epoch [2/5], Loss: 0.9687
Epoch [3/5], Loss: 1.3041
Epoch [4/5], Loss: 1.2549
Epoch [5/5], Loss: 1.0575


In [16]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 56.26 %


# Мodel 1: Accuracy - 63.67 %
Количество слоев: 2 сверточных слоя, 2 макс. объединяющих слоя, 3 полности подключенных слоя,
Фукция активная: RELU,
отступ: 0
размер ядра: 5*5, шагом = 1
Регуляризация: нет
Нормализация: BatchNorm

# Model 2: Accuracy - 60.8 %
Количество слоев: 2 сверточных слоя, 2 макс. объединяющих слоя, 3 полности подключенных слоя,
Фукция активная: RELU,
отступ: 1
размер ядра: 3*3, шагом = 1
Регуляризация: нет
Нормализация: BatchNorm

# Model 3: Accuracy - 56.12 %
Количество слоев: 2 сверточных слоя, 2 макс. объединяющих слоя, 3 полности подключенных слоя,
Фукция активная: RELU,
отступ: 1
размер ядра: 5*5, шагом = 1
Регуляризация: нет
Нормализация: LayerNorm

# Model 4: Accuracy -  54,44 %
Количество слоев: 2 сверточных слоя, 2 макс. объединяющих слоя, 3 полности подключенных слоя,
Фукция активная: RELU,
отступ: 1,
размер ядра: 3*3, шагом = 1,
Регуляризация: 0.25,
Нормализация: BatchNorm,

# Model 5: Accuracy -  57, 58 %
Количество слоев: 6 сверточных слоев, 3 макс. объединяющих слоя, 3 полности подключенных слоя,
Фукция активная: RELU,
отступ: 1,
размер ядра: 3*3, шагом = 1,
Регуляризация: 0.25,
Нормализация: нет,

# Model 6: Accuracy -  58 %
Количество слоев: 6 сверточных слоев, 3 макс. объединяющих слоя, 3 полности подключенных слоя,
Фукция активная: RELU,
отступ: 1,
размер ядра: 3*3, шагом = 1,
Регуляризация: 0.25,
Нормализация: BatchNorm,